# Load Zarr Image from a public S3 store and analyze it in parallel

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/omero-guide-python/blob/master/notebooks/zarr-public-s3-segmentation-parallel.ipynb)

The images are taken from  the paper "In Toto Imaging and Reconstruction of Post-Implantation Mouse Development at the Single-Cell Level" published October 2018 in Cell: https://doi.org/10.1016/j.cell.2018.09.031. The images can be viewed online in the [Image Data Resource](http://idr.openmicroscopy.org/webclient/?show=project-502).

Some images from the Image Data Resource (IDR) have been converted into Zarr using [bioformats2raw](https://github.com/glencoesoftware/bioformats2raw) and stored on a public S3 store.

What is **Zarr**? [Zarr](https://zarr.readthedocs.io/en/stable/) is a young format for the storage of chunked, compressed, N-dimensional arrays.

We will use [Dask](https://docs.dask.org/en/latest/), a flexible library for parallel computing in Python. 

### Install dependencies if required
The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). 

In [ ]:
# Package to access data on S3
%pip install aiohttp==3.9.5 zarr==2.17.2 dask-image==2023.8.1

### Import 

In [1]:
import dask
import dask.array as da
import dask_image.ndfilters
import dask_image.ndmeasure

import matplotlib.pyplot as plt
%matplotlib inline

### ID of the IDR image to analyze
The original image ID is used to identify the file on the S3 store.

In [2]:
image_id = 4007801

### Load Zarr Image from S3

The method will return a dask array without any binary data. The dimension order is ``(TCZYX)``. Data will be loaded when requested during the analysis.

In [3]:
def load_binary_from_s3(id, resolution='4'):
    endpoint_url = 'https://uk1s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    return da.from_zarr(endpoint_url + root)

In [4]:
# Load the dask array
data = load_binary_from_s3(image_id)

In [5]:
# Check the dimension of the array
print(data.shape)

(532, 2, 988, 128, 135)


### Segment the image
This could be replaced by the analysis you wish to perform on a plane.

In [6]:
def analyze(t, c, z):
    plane = data[t, c, z, :, :] 
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
    name = "t:%s, c: %s, z:%s" % (t, c, z)
    return label_image, name

### Use Dask Delayed 

We use ``dask.delayed`` on our function so it records what we want to compute as a task into a graph that will run later on parallel hardware.

Due to the size of the image, we only analyze in the context of this notebook a few planes around the middle z-section and middle timepoint for the first channel. In total, we are analyzing 100 planes.

In [7]:
lazy_results = []
middle_z = data.shape[2] // 2
middle_t = data.shape[0] // 2
range_t = 5
range_z = 5
range_c = 1
for t in range(middle_t - range_t, middle_t + range_t):
    for z in range(middle_z - range_z, middle_z + range_z):
        for c in range(range_c):
            lazy_result = dask.delayed(analyze)(t, c, z)
            lazy_results.append(lazy_result)
print(lazy_results)

[Delayed('analyze-72d81e9b-acc4-430e-a653-588ac70a538c'), Delayed('analyze-c32055a6-80f0-403f-b3fc-b329b567c50e'), Delayed('analyze-9a60668a-0f71-4e02-8a13-2c6801140f53'), Delayed('analyze-c9764adc-c36d-487c-907c-15cbef66befa'), Delayed('analyze-45bcbac6-d3c5-4fed-b5dc-65600107d2de'), Delayed('analyze-0ae6f934-30d6-4600-846a-24eadf990de3'), Delayed('analyze-c5068243-01f1-4e8e-b267-86c5d0bf9b40'), Delayed('analyze-66472bda-a471-4c16-9542-b99b45a6b955'), Delayed('analyze-73bf6e00-1436-4ddf-a06e-a84837b6b881'), Delayed('analyze-8c697e1c-01ea-4624-99a3-f9f99ee46e95'), Delayed('analyze-311f1a60-df41-4b44-95ab-ad6ef71136dc'), Delayed('analyze-4eb87cc5-b421-4ae5-b31a-607bac371abe'), Delayed('analyze-7b3d959e-ed32-4fc2-8b40-cc4f09b928a0'), Delayed('analyze-ce83b0c4-f047-4154-ad83-6d4a5e40726d'), Delayed('analyze-deb21d40-ecbd-4a8a-b0aa-852c08336cab'), Delayed('analyze-5acf941a-b15b-48dd-ac41-b57e3530d152'), Delayed('analyze-2c2a9ece-2628-4505-9124-0f9936807a0e'), Delayed('analyze-1a8c3ca2-83a0

### Run in parallel

The lazy_results list contains information about ``number_t*number_z*number_c`` calls to our function ``analyze`` that have not yet been run. We call ``dask.compute`` when we want the results.
The binary data is loaded from the S3 store during the ``compute`` phase and the analysis is performed.

In [8]:
%time results = dask.compute(*lazy_results)

CPU times: user 1min 32s, sys: 2.82 s, total: 1min 35s
Wall time: 1min 35s


### Display the results

Display the segmented planes. Use the slider to select the plane.

In [9]:
from ipywidgets import *

def display(i=0):
    r, name = results[i]
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(r)
    plt.title(name)
    fig.canvas.flush_events()

interact(display, i= widgets.IntSlider(value=0, min=0, max=len(results)-1, step=1, description="Select Plane", continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Plane', max=99), Output(…

<function __main__.display(i=0)>

![Overview](./images/idr0044_segmented.png)

### License (BSD 2-Clause)
Copyright (C) 2020-20224 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.